# Graphical User Interface for Single Tello Operation 
## Featuring Automated Target Detection and Tracking



### Imports and Globals
The first step is to import the required libraries...

In [1]:
# Standard library imports
import threading 
import socket
import queue
#from PIL import Image
from time import sleep


# Third party imports
import numpy as np
import cv2
from IPython.display import display
import ipywidgets.widgets as widgets

# Local application imports
from tello_camera import TelloCamera
from tello import Tello
from NVidia.object_detection import *

In [2]:
# Globals

#TelloIP = '10.10.30.31'
TelloIP = '192.168.10.1'

TelloCmdPort = 8889      # Command and response
TelloStatusPort = 8890   # Status data from the Tello 
TelloVideoPort = 11111   # h.264 video stream from the Tello

### Create a Graphical User Interface
The user intreface is built using Jupyter Widgets...

Explain event driven programming using on submit and observe methods...

In [3]:
# The status data area is created using a grid widget. This widget allows all of the elements to be placed at uniformly
# spaced intervals. A list of Text widgets is created (status_items) to hold each piece of status data from the Tello.
# One widget from the list is then assigned a specific spot on the grid to create the data layout that is desired.
# The status thread will directly update the values for each widget in the background. The order of the data in the list
# is just the prder defined the the SDK. For example status_item[15] is the battery voltage.

status_grid = widgets.GridspecLayout (9,4)

sg_style = {'description_width':'40px'}

# Create a text box for each of the status items sent from the Tello
status_items = [ widgets.Text(value = str(index),
                              description=str(index),
                              layout=widgets.Layout(width='100px'),
                              style=sg_style, 
                              disabled=True) for index in range(21) ]

#status_grid[0,0] = widgets.Label(value="Found MP:")
status_grid[0,0] = status_items[0]
                          
for i in [1,2,3]:
    status_grid[0,i] = status_items[i]

status_grid[2,0] = widgets.Label(value="Attitude:")
for i in [1,2,3]:
    status_grid[2,i] = status_items[i+4]
    
status_grid[3,0] = widgets.Label(value="Velocity:")
for i in [1,2,3]:
    status_grid[3,i] = status_items[i+7]

status_grid[4,0] = widgets.Label(value="Acc:")
for i in [1,2,3]:
    status_grid[4,i] = status_items[i+17]

status_grid[5,0] = widgets.Label(value="Temp:")
status_grid[5,1] = status_items[11]
status_grid[5,2] = status_items[12]
    
status_grid[6,0] = widgets.Label(value="Time:")
status_grid[6,1] = status_items[13]
status_grid[6,2] = status_items[17]

status_grid[7,0] = widgets.Label(value="Altitdue:")
status_grid[7,1] = status_items[14]
status_grid[7,2] = status_items[16] 
    
status_grid[8,0] = widgets.Label(value="Battery:")
status_grid[8,1] = status_items[15]


In [4]:
# create system status indicators
ss_style = {'description_width':'110px'}
ss_title_bar = widgets.Text(value = TelloIP, description = 'IP Address:')
ss_command_socket = widgets.Valid(value=False, description='Command Socket', style=ss_style)
ss_status_socket = widgets.Valid(value=False, description='Status Socket', style=ss_style)
ss_command_mode = widgets.Valid(value=False, description='Command State', style=ss_style)
ss_video_stream = widgets.Valid(value=False, description='Video Streaming', style=ss_style)
ss_dnn_model_load = widgets.Valid(value=False, description='DNN Model', style=ss_style)

# create a composite control
system_status= widgets.VBox([ss_title_bar,
                             ss_command_socket,
                             ss_status_socket,
                             ss_command_mode,
                             ss_video_stream,
                             ss_dnn_model_load], 
                            layout=widgets.Layout(align_self='auto'))

In [5]:
# create flight control buttons
fc_button_layout = widgets.Layout(width='80px', height='60px', align_self='center')
fc_center_button = widgets.Button(description='Takeoff', button_style='success', layout=fc_button_layout)
fc_forward_button = widgets.Button(description='forward', layout=fc_button_layout)
fc_backward_button = widgets.Button(description='backward', layout=fc_button_layout)
fc_left_button = widgets.Button(description='left', layout=fc_button_layout)
fc_right_button = widgets.Button(description='right', layout=fc_button_layout)
fc_middle_box = widgets.HBox([fc_left_button, fc_center_button, fc_right_button], layout=widgets.Layout(align_self='center'))
fc_button_box = widgets.VBox([fc_forward_button, fc_middle_box, fc_backward_button])

fc_altitude_slider = widgets.IntSlider(
                    value=7,
                    min=20,
                    max=500,
                    step=10,
                    description='Alt:',
                    disabled=False,
                    continuous_update=False,
                    orientation='vertical',
                    readout=True,
                    readout_format='d'
                )

# create a composite control
flight_controls = widgets.HBox([fc_button_box, fc_altitude_slider], layout=widgets.Layout(align_self='center'))

In [6]:
# create a live video window
# Tello's native video resolution is 720p 1280 x 720
video_frame = widgets.Image(format='jpeg', width=170, height=90, description='test')
video_frame = video_frame.from_file('test_image.jpg')

In [7]:
# create target controls
target_selection = widgets.Dropdown(
                        options=[('None', 0), ('Person', 1), ('Cup', 41), ('Three', 3)],
                        value=0,
                        description='Target:',
                        )

detect_button = widgets.Button(description='Detect', layout=widgets.Layout(align_self='center'))
track_button = widgets.Button(description='Track', layout=widgets.Layout(align_self='center'))
target_activation = widgets.HBox([detect_button, track_button])

detection_accuracy = widgets.IntProgress(
                    value=0.0,
                    min=0.0,
                    max=1.0,
                    step=1,
                    description='Accuracy:',
                    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
                    orientation='horizontal'
                )
target_controls = widgets.VBox([target_selection,  
                                detection_accuracy,
                                target_activation] )

In [8]:
#create machine learning controls
epochs_widget = widgets.IntText(description='epochs', value=1, layout=widgets.Layout(align_self='center'))
eval_button = widgets.Button(description='evaluate', layout=widgets.Layout(align_self='center'))
train_button = widgets.Button(description='train')
loss_widget = widgets.FloatText(description='loss')
accuracy_widget = widgets.FloatText(description='accuracy')
progress_widget = widgets.FloatProgress(min=0.0, max=1.0, description='progress')

machine_learning_controls = widgets.VBox([ epochs_widget,
                                           eval_button,
                                           loss_widget,
                                           accuracy_widget,
                                           progress_widget ])

In [9]:
# create command area controls
command_line = widgets.Text(description='Commands:',
                            value='',
                            placeholder='command takeoff land flip forward back left right up down cw ccw speed speed?',
                            layout = widgets.Layout(width='800px'))
# response line
response_line = widgets.Textarea(description='Response:',
                                 value='',
                                 layout = widgets.Layout(width='800px'))

command_area = widgets.VBox([ command_line,response_line ])

### Create The Composit GUI
The AppLayout widget is used to organize the individual elements of the GUI. The widget is divided into five 
main areas - a header, a footer and three middle areas - left, center and right. Each of the sections defined above
are assigned to the AppLayout widget and then the composit GUI is displayed.

In [10]:
left = widgets.VBox([status_grid, 
                     system_status], 
                    layout=widgets.Layout(align_self='auto'))

center = widgets.VBox([video_frame, flight_controls])
right = widgets.VBox([target_controls])

composit_gui = widgets.AppLayout(header = None,
                                 left_sidebar = left,
                                 center = center,
                                 right_sidebar = right,
                                 pane_widths = [2,2,2],
                                 pane_heights = ['50px',3,1],
                                 footer = command_area)
display(composit_gui)

AppLayout(children=(VBox(children=(Text(value='', description='Commands:', layout=Layout(width='800px'), place…

### Define Events

In [11]:
# Send any command entered to the Tello
def on_command_entered(widget_triggered):    
    msg = widget_triggered.value
    msg = msg.encode(encoding="utf-8") 
    sent = cmd_socket.sendto(msg, (TelloIP, TelloCmdPort))
    #widget_triggered.value = ''
command_line.on_submit(on_command_entered)


def execute(change):
    image = change['new']
    
    if detect_button.get_state() == :
    
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == 1]
    
    # get detection closest to center of field of view and draw it
    #det = closest_detection(matching_detections)
    #if det is not None:
    #    bbox = det['bbox']
    #    cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    video_frame.value = bytes(cv2.imencode('.jpg', image)[1])

In [12]:
# Create a UDP socket for commands
try:
#    cmd_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#    cmd_socket.bind (('192.168.10.2', TelloCmdPort)) #host ip, port
    tello = Tello('192.168.10.2', TelloCmdPort)
except OSError as msg:
    cmd_socket = None
    ss_command_socket.value = False
    response_line.Value = str(msg)
else:
    ss_command_socket.value = True

    response_line.value = tello.send_command('command')
    if response_line.value == 'ok':
        ss_command_mode.value = True



>> send cmd: command


### Create Tello Status Thread
Create thread to collect status data

In [13]:
# The status data is sent from the Tello in a fixed order. The background thread reads that data and builds a list
# of strings containing the data lables and values. 

def receive_status_thread(_status_items):

    while True: 
        try:
            b_data, ip = status_socket.recvfrom(1518)
            #print ("Got a status", ip, data)
            data = b_data.decode(encoding='utf-8')
            data = data[:-5].split(';') # convert the data into an array of individual strings, cutting off the trailing ';\r\n'

            # The elements defined in the GUI rely on the data being saved to the list in order as defined in the SDK
            # todo - test to make sure data lenght = list size
            for index in range (len(data)):
                _status_items[index].description = data[index].split(':')[0]
                _status_items[index].value = data[index].split(':')[1]              
            
            # Convert the status data byte array to a string and save it and the ip address to the queue
            #StatusQ.put(PacketBlob(data.decode(encoding="utf-8"), ip[0]))
            
        except Exception:
            print ('\nreceive_status exception - Exit . . .\n')
            break

# Create a UDP socket for status
try:
    status_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    status_socket.bind(('192.168.10.2', TelloStatusPort)) # host, port
except OSError as msg:
    ststus_socket = None
    ss_status_socket.value = False
    response_line.value = str(msg)
else:
    status_thread = threading.Thread(target=receive_status_thread, args=(status_items,))
    status_thread.start()
    ss_status_socket.value = True


### Create a camera opbject to receive video from the Tello
The Tello uses WiFi for all of its communication.

The video socket is created by the OpenCV VideoCapture method defined in the TelloCamera class.
*** Important Note: the current Tello firmware can only stream video if the Tello is directly connected to the host 
computer - i.e. at 192.168.10.1 without using a router. If you assign it to an existing netowork it will not stream video.
The streamon command will not return an error.

In [16]:
# When the TelloCaera class is instantiated, it will test for valid video coming from the
# Tello. If there is no active video stream, the camera object will fail.

# start the video stream
response_line.value = tello.send_command('streamon')

# The Tello camera resoultion is (720,960,3) but SSD_mobilenet was trained on an image
# of size (300,300,3). Setting the width and height will force the camera class to 
# resize the image to be compatible with the SDD_mobilenet engine
try:
    camera = TelloCamera.instance(width=300, height=300)
except RuntimeError as msg:
    camera = None
    ss_video_stream.value = False
    response_line.value = str(msg)
else:
    camera.unobserve_all()
    camera.observe(execute, names='value')
    ss_video_stream.value = True
    

>> send cmd: streamon


### Load a Deep Neural Network for Image Processing
The ...

In [15]:
try:
    model = ObjectDetector('ssd_mobilenet_v2_v04_coco.engine')
except:
    response_line.value = "The DNN model failed ot load"
    ss_dnn_model_load.value = False
else:
    ss_dnn_model_load.value = True

### Commands to get started

Place the Tello in command mode by sending 
    command
You should receive an ok. Sometimes the thread misses this - todo - debug this

start the video stream by sending
    streamon
You should receive an ok and video should appear inthe center window. There is a slight lag in the video.

todo - clear the command and response boxes on reentry to the command box